In [ ]:
import numpy as np
import scipy.linalg as splin
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = "100"
plt.rcParams["animation.html"] = "html5"
import time

# length, speed, time
x0 = .2 ; length = 1. ; k = 5. ; speed = 10. ; tf = .5 
n = 1000 # number of points
m = 200  # number of snapshots

def f_steadypulse(x, t):
    return 1+np.exp(-20*(x-x0)**2)*np.cos(2*np.pi*k*t)

def f_oscpulse(x, t):
    return 1+np.exp(-20*(x-length/2)**2)*np.sin(2*np.pi*k*(x-length/2+x0*np.sin(5*k*t)))

def f_convpulse(x, t):
    return 1+np.exp(-20*(x-x0-speed*t)**2)

def f_pulse_per(x, t):
    return 1+np.exp(-10*((x-x0-speed*t))**2)

def f_per(x, t):
    return 1+np.exp(np.sin(2*k/length*np.pi*(x-x0-speed*t)-1.)) 

xx = np.linspace(0., length, n)
tt = np.linspace(0., tf, m)

# build data
snapshots = np.zeros((n,m))
for j,t in enumerate(tt):
    snapshots[:,j] = f_oscpulse(xx, t)

import matplotlib.animation as animation
fig, ax = plt.subplots()
line, = ax.plot(xx, snapshots[:,0])
def animate(i):
    line.set_ydata(snapshots[:,i])  # update the data.
    return line,
ani = animation.FuncAnimation(fig, animate, interval=25, blit=True)
ani


In [ ]:
def pod_svd(snapshots, maxmode=-1):
    # SVD decomposition of A = U.S.V 
    modes, sval, tcoefs = splin.svd(snapshots)  # returns U, S, V
    return modes[:,:maxmode], sval[:maxmode], tcoefs.T[:,:maxmode] # time evolution must be transposed to be columnwise

def pod_cov(snapshots, maxmode=-1):
    # SVD decomposition of A = U.S.V through eigenvalue problem AT.A
    m = snapshots.shape[1]
    cov = np.matmul(snapshots.T, snapshots)    # compute covariance matrice cov = AT.A = VT.S2.V
    sqval, rmode = splin.eigh(cov)             # eigenvalue of symmetric problem cov = L.D.R
    # ineg = np.argwhere(sqval <0.)
    # if ineg.size > 0:
    #   print(f"some bad values (negative square): {np.squeeze(ineg)}")
    idex = np.flip(np.argsort(sqval))          # compute (increasing) order and reverse 
    sval = np.sqrt(abs(sqval[idex]))           # singular values (S) are square root of D (and sort)
    rmode = rmode[:,idex]                      # R = V (and sort)
    modes = np.matmul(snapshots, rmode.T/sval) # U.S = A.VT 
    return modes[:,:maxmode], sval[:maxmode], rmode[:,:maxmode]



In [ ]:
for thispod in [ pod_svd, pod_cov]:
    method = thispod.__name__
    tic = time.perf_counter()
    modes, sval, tcoefs = thispod(snapshots-np.average(snapshots), maxmode=40)
    toc = time.perf_counter()
    print(sval[:10])
    print(f"process time of pod {method} is {toc-tic:.3e} seconds")
    fig, ax = plt.subplots(4, 2, figsize=(10,12))
    fig.suptitle(method)
    ax[0,0].semilogy(sval, 'o',markersize=2)
    ax[0,1].plot(xx, snapshots[:,0])
    for i in range(3):
        ax[i+1,0].plot(xx, modes[:,i])
        ax[i+1,1].plot(tt, tcoefs[:,i])
    plt.show()
    #plt.hold()